In [1]:
#Import dependencies
from sqlalchemy import create_engine
import pandas as pd
import os
import sqlite3
from pathlib import Path

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [2]:
# File paths
accidents_path = Path("data/Accidents.csv")
drivers_path = Path("data/Licensed_Drivers_clean.csv")
database_path = Path("data/traffic_accidents.sqlite")

In [3]:
# Load CSV files into DataFrames
accidents_df = pd.read_csv(accidents_path)
drivers_df = pd.read_csv(drivers_path)

In [4]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect(database_path)

In [5]:

# Write DataFrames to SQLite
accidents_df.to_sql("accidents", conn, if_exists="replace", index=False)
drivers_df.to_sql("licensed_drivers", conn, if_exists="replace", index=False)

50

In [6]:
# Verify tables
print("Tables created in SQLite database:")
print(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn))

Tables created in SQLite database:
               name
0         accidents
1  licensed_drivers


In [7]:
# Preview data from both tables
print("\nAccidents Table:")
print(pd.read_sql_query("SELECT * FROM accidents LIMIT 5;", conn))


Accidents Table:
      ID  Severity           Start_Time             End_Time  Start_Lat  \
0  A-100         2  2016-02-11 08:13:24  2016-02-11 08:43:24  39.749916   
1  A-200         2  2016-02-16 11:10:17  2016-02-16 11:40:17  39.791660   
2  A-300         2  2016-02-19 16:16:51  2016-02-19 17:01:51  39.765259   
3  A-400         3  2016-02-26 16:15:10  2016-02-26 16:45:10  39.914680   
4  A-500         2  2016-03-04 14:55:55  2016-03-04 15:40:55  39.747753   

   Start_Lng                                        Description  \
0 -84.139359  Accident on US-35 Westbound at Smithville Rd. ...   
1 -84.169342  Accident on Kuntz Rd at Janney Rd. Expect delays.   
2 -84.137802               Accident on Smithville Rd at 3rd St.   
3 -83.016907      Accident on I-71 at Exit 104 OH-104 Frank Rd.   
4 -84.205582                         Accident on US-35 at I-75.   

          Street      City      County  ... Railway Roundabout Station Stop  \
0        US-35 E    Dayton  Montgomery  ...      

In [8]:
print("\nLicensed Drivers Table:")
print(pd.read_sql_query("SELECT * FROM licensed_drivers LIMIT 5;", conn))


Licensed Drivers Table:
  State     Drivers
0    AK    536033.0
1    AL   3999057.0
2    AR   2145334.0
3    AZ   5284970.0
4    CA  27039400.0


In [9]:
# Close the connection
conn.close()

In [10]:
# Create a SQLAlchemy engine
engine = create_engine("sqlite:///data/traffic_accidents.sqlite")

In [11]:
# Read the 'accidents' table
accidents_query = "SELECT * FROM accidents"
accidents_df = pd.read_sql(accidents_query, engine)
print("Accidents Table:")
print(accidents_df.head())

Accidents Table:
      ID  Severity           Start_Time             End_Time  Start_Lat  \
0  A-100         2  2016-02-11 08:13:24  2016-02-11 08:43:24  39.749916   
1  A-200         2  2016-02-16 11:10:17  2016-02-16 11:40:17  39.791660   
2  A-300         2  2016-02-19 16:16:51  2016-02-19 17:01:51  39.765259   
3  A-400         3  2016-02-26 16:15:10  2016-02-26 16:45:10  39.914680   
4  A-500         2  2016-03-04 14:55:55  2016-03-04 15:40:55  39.747753   

   Start_Lng                                        Description  \
0 -84.139359  Accident on US-35 Westbound at Smithville Rd. ...   
1 -84.169342  Accident on Kuntz Rd at Janney Rd. Expect delays.   
2 -84.137802               Accident on Smithville Rd at 3rd St.   
3 -83.016907      Accident on I-71 at Exit 104 OH-104 Frank Rd.   
4 -84.205582                         Accident on US-35 at I-75.   

          Street      City      County  ... Railway Roundabout Station Stop  \
0        US-35 E    Dayton  Montgomery  ...       

In [12]:
# Read the 'licensed_drivers' table
drivers_query = "SELECT * FROM licensed_drivers"
drivers_df = pd.read_sql(drivers_query, engine)
print("\nLicensed Drivers Table:")
print(drivers_df.head())


Licensed Drivers Table:
  State     Drivers
0    AK    536033.0
1    AL   3999057.0
2    AR   2145334.0
3    AZ   5284970.0
4    CA  27039400.0


In [13]:
# SQL query to count accidents per city
accidents_query = """
SELECT City, COUNT(*) AS num_accidents
FROM accidents
GROUP BY City
"""

# Execute the query and load it into a DataFrame
accidents_df = pd.read_sql(accidents_query, engine)

# Display the result
print(accidents_df.head())

         City  num_accidents
0           0              3
1  Aaronsburg              1
2   Abbeville              4
3      Abbott              1
4    Aberdeen             24


In [14]:
# Load accidents table
accidents_df = pd.read_sql("SELECT * FROM accidents", engine)

# Define a list of conditions to look for in the Weather_Condition column
conditions = [
    'Wind', 'Thunderstorm', 'Blowing', 'T-Storm', 
    'Funnel Cloud', 'Squalls', 'Sand', 'Dust'
]

# Create a mask to filter rows where 'Weather_Condition' contains any of the specified conditions
condition_mask = accidents_df['Weather_Condition'].str.contains('|'.join(conditions), case=False, na=False)

# Filter the accidents DataFrame to only include rows with the specified weather conditions
filtered_df = accidents_df[condition_mask]

# Count accidents by each weather condition
accidents_per_condition = filtered_df['Weather_Condition'].value_counts().reset_index()
accidents_per_condition.columns = ['Weather_Condition', 'Accident_Count']

# Display the result
print(accidents_per_condition)

               Weather_Condition  Accident_Count
0                   Fair / Windy             338
1                        T-Storm             171
2                 Cloudy / Windy             166
3          Mostly Cloudy / Windy             156
4          Partly Cloudy / Windy             108
5                  Heavy T-Storm              89
6             Light Rain / Windy              88
7             Light Snow / Windy              70
8   Light Thunderstorms and Rain              51
9                   Thunderstorm              42
10  Heavy Thunderstorms and Rain              20
11                  Rain / Windy              19
12                  Haze / Windy              16
13        Thunderstorms and Rain              15
14                  Snow / Windy              15
15            Heavy Rain / Windy              12
16          Blowing Snow / Windy              11
17                  Blowing Snow              10
18            Heavy Snow / Windy              10
19               Thu